In [17]:
import libpypack
from libpypack.locations import map_locations
from libpypack.visualization import generate_maps
from libpypack.visualization import heatmap
from libpypack.visualization import choropleth

import gmplot
import docker
import requests
import tarfile
import os
import geopandas

In [43]:
import pandas as pd
# df = pd.read_csv("/Users/thatcher/Documents/GraduateSchool/Thesis/FORKS/PyPACK/test_data/Swing_States.csv")
# df

df=pd.read_csv("/Users/thatcher/Documents/GraduateSchool/Thesis/FORKS/PyPACK/test_data/Swing_States.csv", sep=',', nrows=0)
x = list(df.columns.values)
x




['Text', 'URLs']

In [19]:
tweet_df = map_locations.locations_df("/Users/thatcher/Documents/GraduateSchool/Thesis/FORKS/PyPACK/test_data/Swing_States.csv", output_dir="/Users/thatcher/Desktop", df_column="Text", sep=' ')
# over_gdf = generate_maps.generate_overlay_gdf(tweet_df, filename="/Users/thatcher/Documents/GraduateSchool/Thesis/FORKS/PyPACK/libpypack/examples/states_21basic/states.shp")
# generate_maps.plot_gdf(over_gdf)
# heatmap.heatmap(loc_gdf)
# choropleth.choropleth_map(loc_gdf, shp_path="/Users/thatcher/Documents/GraduateSchool/Thesis/FORKS/PyPACK/libpypack/examples/states_21basic/states.shp")

In [22]:
loc_gdf = generate_maps.get_loc_gdf(tweet_df)

In [33]:
import folium
import os
from folium.plugins import HeatMap


def heatmap(loc_gdf, column="Location Extracted", heat_value=None, normalize_data=False, output_name='heatmap.html', output_dir=''):
    locations = loc_gdf[column].value_counts().values
    max_amount = float(locations.max())

    if(normalize_data):
        locations = loc_gdf['Location Extracted'].value_counts().values
        print(locations)
        normalized = (locations-locations.min())/(locations.max()-locations.min())

        hmap = folium.Map(zoom_start=7)

        hm_wides = HeatMap( list(zip(loc_gdf.Latitude.values, loc_gdf.Longitude.values, normalized)),
                           min_opacity=0.2,
                           radius=17, blur=15,
                           max_zoom=1,
                         )

        hmap.add_child(hm_wides)
        hmap.save(os.path.join(output_dir, output_name))

        return hmap

    else:

        max_amount = float(locations.max())

        hmap = folium.Map(zoom_start=7)
        print(list(zip(loc_gdf.Latitude.values, loc_gdf.Longitude.values, locations)))
        hm_wides = HeatMap( list(zip(loc_gdf.Latitude.values, loc_gdf.Longitude.values)),
                           min_opacity=0.2,
                           max_val=max_amount,
                           radius=17, blur=15,
                           max_zoom=1,
                         )

        hmap.add_child(hm_wides)
        hmap.save(os.path.join(output_dir, output_name))

        return hmap


heat = heatmap(loc_gdf)
heat

[(42.00027, -93.50049, 1), (44.25029, -85.50033, 1), (46.25024, -94.25055, 1), (40.25034, -83.00018, 1), (39.25021, -116.75119, 1), (43.66702, -71.4998, 1), (35.50069, -80.00032, 1), (40.27245, -76.90567, 1), (44.50024, -90.00041, 1)]


In [ ]:
choropleth.choropleth_map(loc_gdf, shp_path="/Users/thatcher/Documents/GraduateSchool/Thesis/FORKS/PyPACK/libpypack/examples/states_21basic/states.shp")

In [10]:
heat

NameError: name 'heattttmap' is not defined